In [1]:
# Thanks to Nicks Niche for the starter code
# in case needed
#!pip install pulp

In [2]:
# Considerations
# - Players are not independent, how to account for stacks, correlations?
# - We are playing against others and have to share the winnings, how to get slight edge?
# -- maybe cross off top picks by position
# -- Limit total salary (eg everyone trying to spend 50k, maybe spend 48)
# - We are passing the mean score, but need to consider variance. Might want more risk to get ahead.
# -- Could vary the projections by x%

In [3]:
import pandas as pd
import numpy as np
from pulp import *

from src.utils import fix_name_mismatches
from src.components.import_data import ImportData
from src.components.optimizer import OptimizeLineup

from input_vars import *

### Read in Draft Kings Players/Salaries and Rankings

In [6]:
gd = ImportData(week, game_mode, rank_src, matchup, add_results)
players = gd.read_dk()
player_rankings = gd.read_rankings(players, adhoc_pt_col=adhoc_pt_col)

In [7]:
# QA check for any large names missing rank data
# May signal a name mismatch across sources
mask = player_rankings['FPPG'].isnull() #& player_rankings['Salary'] > 500
player_rankings[mask].sort_values('Salary', ascending=False).head(5)

,Id,Position,Salary,FPPG,ActualFP
83,Tyreek Hill,WR,9900,NaN,NaN
496,Tyreek Hill,FLEX,9900,NaN,NaN
4,C.J. Stroud,QB,7100,NaN,NaN
512,Chris Olave,FLEX,7100,NaN,NaN
99,Chris Olave,WR,7100,NaN,NaN


In [8]:
availables = player_rankings.groupby(['Position', 'Id', 'FPPG', 'Salary']).agg('count')
availables = availables.reset_index()

# screening on availables
#mask = availables['Salary'] > 900
#availables = availables[mask]

availables = availables[~availables['Id'].isin(scratches)].reset_index(drop=True)

#check for any force ins not in list
print(f"Names Not Found: {list(set(force_ins) - set(availables['Id']))}")

Names Not Found: ['Drew Lock']


In [9]:
availables

,Position,Id,FPPG,Salary,ActualFP
0,DST,49ers,8.0,4400,0
1,DST,Bears,6.1,3300,0
2,DST,Bills,5.6,2400,0
3,DST,Browns,7.6,3500,0
4,DST,Buccaneers,5.5,2900,0
...,...,...,...,...,...
511,WR,Velus Jones Jr.,1.0,3000,0
512,WR,Wan'Dale Robinson,9.5,3700,0
513,WR,Xavier Gipson,5.1,3200,0
514,WR,Xavier Hutchinson,4.7,3000,0


### Run Optimizer

In [10]:
op = OptimizeLineup(game_mode, availables, n_lineups, force_ins, add_results, variability_pctg)
results = op.run_optimizer()
results.head(20).T

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Jets_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_
2,FLEX_Tucker_Kraft,FLEX_Ezekiel_Elliott,FLEX_Demarcus_Robinson,FLEX_Christian_McCaffrey,FLEX_Christian_McCaffrey,FLEX_Kyren_Williams,FLEX_Christian_McCaffrey,FLEX_Ezekiel_Elliott,FLEX_Christian_McCaffrey,FLEX_Ezekiel_Elliott,FLEX_Kyren_Williams,FLEX_Kyren_Williams,FLEX_George_Kittle,FLEX_Antonio_Gibson,FLEX_Kyren_Williams,FLEX_Ezekiel_Elliott,FLEX_Kyren_Williams,FLEX_Darius_Slayton,FLEX_Kyren_Williams,FLEX_Kyren_Williams
3,QB_Matthew_Stafford,QB_Matthew_Stafford,QB_Josh_Allen,QB_Matthew_Stafford,QB_Brock_Purdy,QB_Baker_Mayfield,QB_Matthew_Stafford,QB_Josh_Allen,QB_Josh_Allen,QB_Matthew_Stafford,QB_Josh_Allen,QB_Josh_Allen,QB_Brock_Purdy,QB_Matthew_Stafford,QB_Zach_Wilson,QB_Matthew_Stafford,QB_Brock_Purdy,QB_Matthew_Stafford,QB_Matthew_Stafford,QB_Brock_Purdy
4,RB_Christian_McCaffrey,RB_Christian_McCaffrey,RB_Christian_McCaffrey,RB_Ezekiel_Elliott,RB_Ezekiel_Elliott,RB_Christian_McCaffrey,RB_Ezekiel_Elliott,RB_Christian_McCaffrey,RB_Ezekiel_Elliott,RB_Christian_McCaffrey,RB_Ezekiel_Elliott,RB_Christian_McCaffrey,RB_Ezekiel_Elliott,RB_Christian_McCaffrey,RB_Bijan_Robinson,RB_Christian_McCaffrey,RB_Christian_McCaffrey,RB_Christian_McCaffrey,RB_Chuba_Hubbard,RB_Christian_McCaffrey
5,RB_Kyren_Williams,RB_Kyren_Williams,RB_Kyren_Williams,RB_Kyren_Williams,RB_Kyren_Williams,RB_Ezekiel_Elliott,RB_Kyren_Williams,RB_Kyren_Williams,RB_Kyren_Williams,RB_Kyren_Williams,RB_Tony_Pollard,RB_Ezekiel_Elliott,RB_Kyren_Williams,RB_Kyren_Williams,RB_Christian_McCaffrey,RB_Derrick_Henry,RB_Derrick_Henry,RB_Kyren_Williams,RB_Ezekiel_Elliott,RB_Ezekiel_Elliott
6,TE_Trey_McBride,TE_Trey_McBride,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Tucker_Kraft,TE_Hunter_Henry,TE_Trey_McBride,TE_Tucker_Kraft
7,WR_DeAndre_Hopkins,WR_Dontayvion_Wicks,WR_DeAndre_Hopkins,WR_Demarcus_Robinson,WR_DeAndre_Hopkins,WR_Garrett_Wilson,WR_Garrett_Wilson,WR_Demarcus_Robinson,WR_Jayden_Reed,WR_Demarcus_Robinson,WR_DeAndre_Hopkins,WR_Demarcus_Robinson,WR_DeAndre_Hopkins,WR_Dontayvion_Wicks,WR_DeAndre_Hopkins,WR_DeAndre_Hopkins,WR_Demario_Douglas,WR_DeAndre_Hopkins,WR_Jayden_Reed,WR_Darius_Slayton
8,WR_Demario_Douglas,WR_Jalin_Hyatt,WR_Jalin_Hyatt,WR_Garrett_Wilson,WR_Demario_Douglas,WR_Jayden_Reed,WR_Jayden_Reed,WR_Jayden_Reed,WR_Michael_Wilson,WR_Garrett_Wilson,WR_Jayden_Reed,WR_Jayden_Reed,WR_Garrett_Wilson,WR_Jayden_Reed,WR_Jalin_Hyatt,WR_Jayden_Reed,WR_Jayden_Reed,WR_Garrett_Wilson,WR_Puka_Nacua,WR_Jayden_Reed
9,WR_Jayden_Reed,WR_Jayden_Reed,WR_Jayden_Reed,WR_Jayden_Reed,WR_Jayden_Reed,WR_Jonathan_Mingo,WR_Wan'Dale_Robinson,WR_Wan'Dale_Robinson,WR_Wan'Dale_Robinson,WR_Jayden_Reed,WR_Wan'Dale_Robinson,WR_Wan'Dale_Robinson,WR_Jayden_Reed,WR_Rashee_Rice,WR_Jayden_Reed,WR_Jonathan_Mingo,WR_Wan'Dale_Robinson,WR_Jayden_Reed,WR_Wan'Dale_Robinson,WR_Terry_McLaurin
ProjFP,135.059567,135.047743,135.037222,134.685759,134.657599,134.628583,134.423878,134.379867,134.315191,134.121049,134.011424,133.961747,133.951692,133.941601,133.931066,133.91926,133.903296,133.893205,133.882954,133.854596


In [11]:
# Current entry is below
results.loc[2,]

1                  DST_Dolphins_
2           FLEX_Ezekiel_Elliott
3            QB_Matthew_Stafford
4         RB_Christian_McCaffrey
5              RB_Kyren_Williams
6                TE_Trey_McBride
7            WR_Dontayvion_Wicks
8                 WR_Jalin_Hyatt
9                 WR_Jayden_Reed
ProjFP                135.047743
Name: 2, dtype: object

1          CPT_Trevor_Lawrence
2                FLEX_Bengals_
3         FLEX_Brandon_McManus
4           FLEX_Ja'Marr_Chase
5           FLEX_Jake_Browning
6               FLEX_Joe_Mixon
ProjFP               89.356369
Name: 2, dtype: object